In [1]:
import pandas as pd
import numpy as np

from numpy.random import seed
import tensorflow as tf
import random as rn
import os
from nltk import tokenize 
import nltk
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
nltk.download('punkt')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
CODES_PATH = 'drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-classification'
CV_PATH    = 'drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-classification/models/cv'
TEST_PATH  = 'drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-classification/models/final'

In [3]:
max_features = 7000
maxlen       = 2000
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150


seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [4]:
import sys
sys.path.append(f'{CODES_PATH}')

In [5]:
if tf.test.is_gpu_available():
  import models
else:
  print('You need the GPU support for these calculations')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def other(x):
    if ((x=='exchanges')|(x=='finance')|(x=='games')|(x=='gambling')|(x=='high-risk')): 
        return x 
    else: 
        return 'other'

In [7]:
data = pd.read_csv("drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-Dapp-scraping/data/dapps_source_code_parsed_open_source_2021-01-08.csv")

data = data[data.SC_full!='\n']
data.drop('Unnamed: 0', axis=1, inplace=True)
data.category = data.category.apply(other)
data.comments.fillna('empty', inplace=True)

In [8]:
RS = 42
SPLITS = 3
LENGTH = 5000

In [9]:
X = data.drop('category', axis=1)
y = data.category

In [10]:
X, X_test, y, y_test  = train_test_split(X, y, stratify=y, test_size=0.2, random_state=RS)

In [11]:
print(f'X_train shape {X.shape}')
print(f'X_test shape {X_test.shape}')

X_train shape (1142, 12)
X_test shape (286, 12)


## FULL

In [12]:
names  = [f'aucroc_{i}' for i in ['exchanges', 'finance', 'gambling', 'games', 'high-risk']]
names2 = [f'aucprc_{i}' for i in ['exchanges', 'finance', 'gambling', 'games', 'high-risk']]
names.extend(names2)

kf  = StratifiedKFold(n_splits=SPLITS, random_state=RS, shuffle=True)
TRAIN = True

In [13]:
n_words  = range(500, 1200, 200)
length   = range(500, 1200, 200)
emb_dim  = [50, 100]
n_hidden = [50, 100]
epochs   = [4]
lr       = [0.01, 0.001, 0.0001]
drop     = [0, 0.25]


n_words  = [1100]
length   = [500]
emb_dim  = [50]        
n_hidden = [100]      
drop     = [0]      
epochs   = [4]
lr       = [0.01]

### BGRU

In [14]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['SC_full'], X.iloc[val_index]['SC_full']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='BGRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/BGRU_full_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/BGRU_full_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 7s 423ms/step - loss: 1.3042 - acc: 0.3727
Epoch 2/4
6/6 [==============================] - 0s 59ms/step - loss: 1.0859 - acc: 0.5040
Epoch 3/4
6/6 [==============================] - 0s 59ms/step - loss: 0.9591 - acc: 0.5417
Epoch 4/4
6/6 [==============================] - 0s 56ms/step - loss: 0.9668 - acc: 0.5518
Epoch 1/4
6/6 [==============================] - 2s 78ms/step - loss: 1.3182 - acc: 0.3543
Epoch 2/4
6/6 [==============================] - 0s 58ms/step - loss: 1.0397 - acc: 0.5280
Epoch 3/4
6/6 [==============================] - 0s 57ms/step - loss: 1.6120 - acc: 0.5095
Epoch 4/4
6/6 [==============================] - 0s 57ms/step - loss: 0.8970 - acc: 0.5182
Epoch 1/4
6/6 [==============================] - 4s 452ms/step - loss: 1.2854 - acc: 0.3731
Epoch 2/4
6/6 [==============================] - 0s 59ms/step - loss: 1.0594 - acc: 0.5149
Epoch 3/4
6/6 [==============================] - 0s 60ms/step - loss: 1.0118 - acc: 0.51

In [15]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.915,0.730,0.668,0.774,0.691,0.912,0.494,0.079,0.424,0.117
1,0.924,0.829,0.685,0.830,0.820,0.922,0.691,0.080,0.425,0.147
2,0.949,0.820,0.735,0.840,0.766,0.945,0.656,0.098,0.541,0.142


In [16]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['SC_full'], X_test['SC_full']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='BGRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/BGRU_full_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/BGRU_full_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 4s 305ms/step - loss: 1.2597 - acc: 0.4106
Epoch 2/4
9/9 [==============================] - 1s 57ms/step - loss: 1.0992 - acc: 0.4961
Epoch 3/4
9/9 [==============================] - 1s 57ms/step - loss: 0.9611 - acc: 0.5853
Epoch 4/4
9/9 [==============================] - 1s 57ms/step - loss: 0.9899 - acc: 0.5135


In [17]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.97,0.848,0.767,0.795,0.86,0.963,0.699,0.149,0.422,0.256


### GRU

In [18]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['SC_full'], X.iloc[val_index]['SC_full']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='GRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/GRU_full_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/GRU_full_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 1s 49ms/step - loss: 1.3113 - acc: 0.3699
Epoch 2/4
6/6 [==============================] - 0s 34ms/step - loss: 1.3045 - acc: 0.4920
Epoch 3/4
6/6 [==============================] - 0s 31ms/step - loss: 1.1065 - acc: 0.4683
Epoch 4/4
6/6 [==============================] - 0s 32ms/step - loss: 0.9657 - acc: 0.5751
Epoch 1/4
6/6 [==============================] - 1s 53ms/step - loss: 1.3076 - acc: 0.4077
Epoch 2/4
6/6 [==============================] - 0s 38ms/step - loss: 1.0764 - acc: 0.5073
Epoch 3/4
6/6 [==============================] - 0s 32ms/step - loss: 1.5645 - acc: 0.5367
Epoch 4/4
6/6 [==============================] - 0s 32ms/step - loss: 0.8824 - acc: 0.6069
Epoch 1/4
6/6 [==============================] - 1s 51ms/step - loss: 1.3148 - acc: 0.4363
Epoch 2/4
6/6 [==============================] - 0s 35ms/step - loss: 1.1890 - acc: 0.4793
Epoch 3/4
6/6 [==============================] - 0s 31ms/step - loss: 1.0230 - acc: 0.5578

In [19]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.917,0.708,0.776,0.794,0.800,0.914,0.470,0.185,0.470,0.170
1,0.932,0.769,0.695,0.725,0.737,0.934,0.594,0.082,0.370,0.091
2,0.955,0.744,0.649,0.790,0.644,0.946,0.531,0.075,0.491,0.104


In [20]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['SC_full'], X_test['SC_full']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='GRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/GRU_full_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/GRU_full_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 1s 47ms/step - loss: 1.2821 - acc: 0.4061
Epoch 2/4
9/9 [==============================] - 0s 32ms/step - loss: 1.0778 - acc: 0.5187
Epoch 3/4
9/9 [==============================] - 0s 32ms/step - loss: 1.1164 - acc: 0.5496
Epoch 4/4
9/9 [==============================] - 0s 31ms/step - loss: 0.9262 - acc: 0.5679


In [21]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.966,0.712,0.749,0.818,0.897,0.956,0.443,0.102,0.442,0.486


### CNN

In [22]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['SC_full'], X.iloc[val_index]['SC_full']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='CNN', max_features=WORD, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/CNN_full_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/CNN_full_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 1s 47ms/step - loss: 1.2621 - acc: 0.3682
Epoch 2/4
6/6 [==============================] - 0s 27ms/step - loss: 0.9943 - acc: 0.5979
Epoch 3/4
6/6 [==============================] - 0s 28ms/step - loss: 0.8813 - acc: 0.7287
Epoch 4/4
6/6 [==============================] - 0s 28ms/step - loss: 0.7885 - acc: 0.6153
Epoch 1/4
6/6 [==============================] - 1s 40ms/step - loss: 1.2546 - acc: 0.4055
Epoch 2/4
6/6 [==============================] - 0s 29ms/step - loss: 0.9969 - acc: 0.6071
Epoch 3/4
6/6 [==============================] - 0s 27ms/step - loss: 0.8516 - acc: 0.6567
Epoch 4/4
6/6 [==============================] - 0s 29ms/step - loss: 0.7850 - acc: 0.6584
Epoch 1/4
6/6 [==============================] - 1s 52ms/step - loss: 1.2386 - acc: 0.4024
Epoch 2/4
6/6 [==============================] - 0s 28ms/step - loss: 0.9617 - acc: 0.6078
Epoch 3/4
6/6 [==============================] - 0s 28ms/step - loss: 0.8440 - acc: 0.6926

In [23]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.963,0.895,0.744,0.882,0.783,0.956,0.778,0.110,0.699,0.188
1,0.968,0.897,0.747,0.872,0.801,0.961,0.811,0.139,0.625,0.173
2,0.960,0.927,0.773,0.874,0.861,0.956,0.830,0.142,0.673,0.169


In [24]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['SC_full'], X_test['SC_full']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='CNN', max_features=WORD, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/CNN_full_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/CNN_full_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 1s 42ms/step - loss: 1.1993 - acc: 0.4037
Epoch 2/4
9/9 [==============================] - 0s 27ms/step - loss: 0.9270 - acc: 0.6607
Epoch 3/4
9/9 [==============================] - 0s 28ms/step - loss: 0.7810 - acc: 0.6560
Epoch 4/4
9/9 [==============================] - 0s 27ms/step - loss: 0.7054 - acc: 0.6769


In [25]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.972,0.95,0.804,0.908,0.84,0.972,0.88,0.155,0.672,0.482


## Only The Source Code

In [26]:
names  = [f'aucroc_{i}' for i in ['exchanges', 'finance', 'gambling', 'games', 'high-risk']]
names2 = [f'aucprc_{i}' for i in ['exchanges', 'finance', 'gambling', 'games', 'high-risk']]
names.extend(names2)

kf  = StratifiedKFold(n_splits=SPLITS, random_state=RS, shuffle=True)
TRAIN = True

In [27]:
n_words  = range(500, 1200, 200)
length   = range(500, 1200, 200)
emb_dim  = [50, 100]
n_hidden = [50, 100]
epochs   = [4]
lr       = [0.01, 0.001, 0.0001]
drop     = [0, 0.25]


n_words  = [1100]
length   = [500]
emb_dim  = [50]        
n_hidden = [100]      
drop     = [0]      
epochs   = [4]
lr       = [0.01]

### BGRU

In [28]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['SC_no_comments'], X.iloc[val_index]['SC_no_comments']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='BGRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/BGRU_onlycode_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/BGRU_onlycode_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 2s 82ms/step - loss: 1.3108 - acc: 0.3842
Epoch 2/4
6/6 [==============================] - 0s 56ms/step - loss: 1.0601 - acc: 0.5044
Epoch 3/4
6/6 [==============================] - 0s 56ms/step - loss: 1.1749 - acc: 0.5046
Epoch 4/4
6/6 [==============================] - 0s 57ms/step - loss: 1.1775 - acc: 0.4953
Epoch 1/4
6/6 [==============================] - 2s 74ms/step - loss: 1.3004 - acc: 0.3730
Epoch 2/4
6/6 [==============================] - 0s 59ms/step - loss: 1.1532 - acc: 0.4975
Epoch 3/4
6/6 [==============================] - 0s 57ms/step - loss: 1.1000 - acc: 0.5326
Epoch 4/4
6/6 [==============================] - 0s 58ms/step - loss: 0.7954 - acc: 0.5665
Epoch 1/4
6/6 [==============================] - 2s 75ms/step - loss: 1.3014 - acc: 0.4669
Epoch 2/4
6/6 [==============================] - 0s 55ms/step - loss: 1.1737 - acc: 0.4906
Epoch 3/4
6/6 [==============================] - 0s 55ms/step - loss: 1.1120 - acc: 0.5218

In [29]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.920,0.710,0.734,0.806,0.771,0.915,0.351,0.100,0.429,0.340
1,0.960,0.743,0.656,0.759,0.800,0.953,0.441,0.096,0.364,0.157
2,0.959,0.688,0.628,0.842,0.781,0.954,0.353,0.066,0.521,0.218


In [30]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['SC_no_comments'], X_test['SC_no_comments']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='BGRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/BGRU_onlycode_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/BGRU_onlycode_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 2s 68ms/step - loss: 1.2955 - acc: 0.4380
Epoch 2/4
9/9 [==============================] - 1s 56ms/step - loss: 1.0086 - acc: 0.5103
Epoch 3/4
9/9 [==============================] - 1s 56ms/step - loss: 0.9172 - acc: 0.5198
Epoch 4/4
9/9 [==============================] - 1s 57ms/step - loss: 0.8156 - acc: 0.5714


In [31]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.946,0.282,0.461,0.744,0.763,0.952,0.157,0.049,0.448,0.508


### GRU

In [32]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['SC_no_comments'], X.iloc[val_index]['SC_no_comments']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='GRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/GRU_onlycode_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/GRU_onlycode_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 1s 50ms/step - loss: 1.3185 - acc: 0.3447
Epoch 2/4
6/6 [==============================] - 0s 36ms/step - loss: 1.3785 - acc: 0.5091
Epoch 3/4
6/6 [==============================] - 0s 32ms/step - loss: 1.1418 - acc: 0.5282
Epoch 4/4
6/6 [==============================] - 0s 33ms/step - loss: 0.9638 - acc: 0.5202
Epoch 1/4
6/6 [==============================] - 1s 51ms/step - loss: 1.3114 - acc: 0.4171
Epoch 2/4
6/6 [==============================] - 0s 38ms/step - loss: 1.4327 - acc: 0.4908
Epoch 3/4
6/6 [==============================] - 0s 31ms/step - loss: 1.2223 - acc: 0.4982
Epoch 4/4
6/6 [==============================] - 0s 31ms/step - loss: 1.3330 - acc: 0.4842
Epoch 1/4
6/6 [==============================] - 1s 47ms/step - loss: 1.3291 - acc: 0.3536
Epoch 2/4
6/6 [==============================] - 0s 34ms/step - loss: 1.2444 - acc: 0.5634
Epoch 3/4
6/6 [==============================] - 0s 32ms/step - loss: 1.0334 - acc: 0.5598

In [33]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.902,0.757,0.751,0.739,0.800,0.897,0.589,0.155,0.415,0.166
1,0.922,0.791,0.760,0.784,0.740,0.925,0.616,0.153,0.453,0.151
2,0.947,0.851,0.702,0.817,0.743,0.933,0.684,0.146,0.500,0.121


In [34]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['SC_no_comments'], X_test['SC_no_comments']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='GRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/GRU_onlycode_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/GRU_onlycode_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 2s 45ms/step - loss: 1.3029 - acc: 0.4679
Epoch 2/4
9/9 [==============================] - 0s 31ms/step - loss: 1.0526 - acc: 0.5578
Epoch 3/4
9/9 [==============================] - 0s 32ms/step - loss: 0.9700 - acc: 0.5011
Epoch 4/4
9/9 [==============================] - 0s 32ms/step - loss: 0.8634 - acc: 0.4935


In [35]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.949,0.802,0.779,0.692,0.888,0.952,0.615,0.121,0.299,0.35


### CNN

In [36]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['SC_no_comments'], X.iloc[val_index]['SC_no_comments']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='CNN', max_features=WORD, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/CNN_onlycode_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/CNN_onlycode_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 1s 39ms/step - loss: 1.2807 - acc: 0.3346
Epoch 2/4
6/6 [==============================] - 0s 29ms/step - loss: 1.0215 - acc: 0.6284
Epoch 3/4
6/6 [==============================] - 0s 27ms/step - loss: 0.8575 - acc: 0.6440
Epoch 4/4
6/6 [==============================] - 0s 27ms/step - loss: 0.7998 - acc: 0.6340
Epoch 1/4
6/6 [==============================] - 1s 36ms/step - loss: 1.2128 - acc: 0.5203
Epoch 2/4
6/6 [==============================] - 0s 30ms/step - loss: 0.9805 - acc: 0.6289
Epoch 3/4
6/6 [==============================] - 0s 27ms/step - loss: 0.8355 - acc: 0.5938
Epoch 4/4
6/6 [==============================] - 0s 27ms/step - loss: 0.7618 - acc: 0.6486
Epoch 1/4
6/6 [==============================] - 1s 37ms/step - loss: 1.2193 - acc: 0.4511
Epoch 2/4
6/6 [==============================] - 0s 31ms/step - loss: 0.9657 - acc: 0.5775
Epoch 3/4
6/6 [==============================] - 0s 29ms/step - loss: 0.8588 - acc: 0.6735

In [37]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.936,0.893,0.656,0.886,0.695,0.933,0.761,0.080,0.665,0.102
1,0.962,0.870,0.795,0.847,0.716,0.954,0.754,0.188,0.593,0.112
2,0.949,0.927,0.718,0.829,0.845,0.945,0.835,0.122,0.575,0.135


In [38]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['SC_no_comments'], X_test['SC_no_comments']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='CNN', max_features=WORD, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/CNN_onlycode_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/CNN_onlycode_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 1s 36ms/step - loss: 1.1844 - acc: 0.4456
Epoch 2/4
9/9 [==============================] - 0s 27ms/step - loss: 0.9091 - acc: 0.6312
Epoch 3/4
9/9 [==============================] - 0s 28ms/step - loss: 0.7867 - acc: 0.5897
Epoch 4/4
9/9 [==============================] - 0s 26ms/step - loss: 0.6921 - acc: 0.6473


In [39]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.952,0.895,0.707,0.868,0.781,0.953,0.794,0.1,0.663,0.168


## Only The Comments

In [40]:
names  = [f'aucroc_{i}' for i in ['exchanges', 'finance', 'gambling', 'games', 'high-risk']]
names2 = [f'aucprc_{i}' for i in ['exchanges', 'finance', 'gambling', 'games', 'high-risk']]
names.extend(names2)

kf  = StratifiedKFold(n_splits=SPLITS, random_state=RS, shuffle=True)
TRAIN = True

In [41]:
n_words  = range(500, 1200, 200)
length   = range(500, 1200, 200)
emb_dim  = [50, 100]
n_hidden = [50, 100]
epochs   = [4]
lr       = [0.01, 0.001, 0.0001]
drop     = [0, 0.25]


n_words  = [1100]
length   = [500]
emb_dim  = [50]        
n_hidden = [100]      
drop     = [0]      
epochs   = [4]
lr       = [0.01]

### BGRU

In [42]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['comments'], X.iloc[val_index]['comments']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='BGRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/BGRU_comments_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/BGRU_comments_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 2s 74ms/step - loss: 1.2985 - acc: 0.4242
Epoch 2/4
6/6 [==============================] - 1s 104ms/step - loss: 1.2077 - acc: 0.4876
Epoch 3/4
6/6 [==============================] - 0s 60ms/step - loss: 1.0674 - acc: 0.5774
Epoch 4/4
6/6 [==============================] - 0s 57ms/step - loss: 0.9736 - acc: 0.5552
Epoch 1/4
6/6 [==============================] - 2s 77ms/step - loss: 1.3169 - acc: 0.3481
Epoch 2/4
6/6 [==============================] - 0s 58ms/step - loss: 1.2303 - acc: 0.4964
Epoch 3/4
6/6 [==============================] - 0s 57ms/step - loss: 1.0738 - acc: 0.5309
Epoch 4/4
6/6 [==============================] - 0s 57ms/step - loss: 1.2031 - acc: 0.4966
Epoch 1/4
6/6 [==============================] - 2s 79ms/step - loss: 1.2780 - acc: 0.3942
Epoch 2/4
6/6 [==============================] - 0s 56ms/step - loss: 1.1766 - acc: 0.5012
Epoch 3/4
6/6 [==============================] - 0s 57ms/step - loss: 1.4918 - acc: 0.533

In [43]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.910,0.760,0.704,0.716,0.739,0.906,0.530,0.157,0.332,0.191
1,0.937,0.719,0.728,0.713,0.773,0.940,0.352,0.107,0.315,0.101
2,0.953,0.752,0.623,0.764,0.643,0.951,0.483,0.066,0.408,0.085


In [44]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['comments'], X_test['comments']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='BGRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/BGRU_comments_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/BGRU_comments_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 2s 67ms/step - loss: 1.3038 - acc: 0.3787
Epoch 2/4
9/9 [==============================] - 0s 54ms/step - loss: 1.3344 - acc: 0.5337
Epoch 3/4
9/9 [==============================] - 0s 55ms/step - loss: 0.9597 - acc: 0.5333
Epoch 4/4
9/9 [==============================] - 1s 56ms/step - loss: 0.8441 - acc: 0.5476


In [45]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.971,0.639,0.656,0.853,0.839,0.968,0.276,0.086,0.52,0.361


### GRU

In [46]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['comments'], X.iloc[val_index]['comments']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='GRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/GRU_comments_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/GRU_comments_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 1s 50ms/step - loss: 1.3488 - acc: 0.3716
Epoch 2/4
6/6 [==============================] - 0s 36ms/step - loss: 1.1515 - acc: 0.4850
Epoch 3/4
6/6 [==============================] - 0s 31ms/step - loss: 0.9532 - acc: 0.5133
Epoch 4/4
6/6 [==============================] - 0s 31ms/step - loss: 1.0347 - acc: 0.5231
Epoch 1/4
6/6 [==============================] - 1s 49ms/step - loss: 1.2877 - acc: 0.4356
Epoch 2/4
6/6 [==============================] - 0s 33ms/step - loss: 1.0792 - acc: 0.5153
Epoch 3/4
6/6 [==============================] - 0s 32ms/step - loss: 1.1487 - acc: 0.4769
Epoch 4/4
6/6 [==============================] - 0s 31ms/step - loss: 0.8945 - acc: 0.6171
Epoch 1/4
6/6 [==============================] - 1s 52ms/step - loss: 1.3081 - acc: 0.4149
Epoch 2/4
6/6 [==============================] - 0s 34ms/step - loss: 1.2513 - acc: 0.5191
Epoch 3/4
6/6 [==============================] - 0s 32ms/step - loss: 0.9507 - acc: 0.6017

In [47]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.902,0.729,0.683,0.761,0.800,0.906,0.373,0.081,0.408,0.128
1,0.928,0.854,0.688,0.748,0.802,0.933,0.683,0.105,0.413,0.146
2,0.937,0.725,0.715,0.724,0.781,0.939,0.485,0.103,0.373,0.143


In [48]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['comments'], X_test['comments']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='GRU', max_features=WORD, embed_dim=DIM, rec_units=HID, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/GRU_comments_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/GRU_comments_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 1s 45ms/step - loss: 1.2720 - acc: 0.3958
Epoch 2/4
9/9 [==============================] - 0s 31ms/step - loss: 1.0736 - acc: 0.4879
Epoch 3/4
9/9 [==============================] - 0s 31ms/step - loss: 0.9183 - acc: 0.5276
Epoch 4/4
9/9 [==============================] - 0s 31ms/step - loss: 0.9453 - acc: 0.5944


In [49]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.96,0.804,0.638,0.84,0.855,0.963,0.648,0.107,0.511,0.313


### CNN

In [50]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['comments'], X.iloc[val_index]['comments']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
                        
                        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                        tokenizer.fit_on_texts(X_train)
                        
                        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                        X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                        X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]

                        classifier = models.dl_model(model_type='CNN', max_features=WORD, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                        if TRAIN:
                                classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                                classifier.save_weights(f'{CV_PATH}/CNN_comments_fold_{c}_v1.h5')
                        else: 
                                classifier.load_weights(f'{CV_PATH}/CNN_comments_fold_{c}_v1.h5')
                        probs = classifier.predict(X_val)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
6/6 [==============================] - 1s 39ms/step - loss: 1.2450 - acc: 0.4231
Epoch 2/4
6/6 [==============================] - 0s 30ms/step - loss: 0.9629 - acc: 0.5916
Epoch 3/4
6/6 [==============================] - 0s 28ms/step - loss: 0.8492 - acc: 0.6625
Epoch 4/4
6/6 [==============================] - 0s 27ms/step - loss: 0.7713 - acc: 0.6712
Epoch 1/4
6/6 [==============================] - 1s 39ms/step - loss: 1.2353 - acc: 0.4127
Epoch 2/4
6/6 [==============================] - 0s 29ms/step - loss: 0.9680 - acc: 0.5891
Epoch 3/4
6/6 [==============================] - 0s 27ms/step - loss: 0.8908 - acc: 0.6779
Epoch 4/4
6/6 [==============================] - 0s 27ms/step - loss: 0.8109 - acc: 0.6918
Epoch 1/4
6/6 [==============================] - 1s 34ms/step - loss: 1.2142 - acc: 0.4161
Epoch 2/4
6/6 [==============================] - 0s 29ms/step - loss: 0.9726 - acc: 0.5850
Epoch 3/4
6/6 [==============================] - 0s 27ms/step - loss: 0.8325 - acc: 0.6709

In [51]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.924,0.876,0.701,0.877,0.723,0.928,0.741,0.085,0.709,0.091
1,0.953,0.894,0.787,0.867,0.767,0.942,0.792,0.137,0.572,0.103
2,0.944,0.881,0.739,0.819,0.749,0.933,0.738,0.100,0.574,0.100


In [52]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

                X_train, X_val      = X['comments'], X_test['comments']
                y_train, y_val      = y, y_test
                
                tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=WORD)
                tokenizer.fit_on_texts(X_train)
                
                list_tokenized_train = tokenizer.texts_to_sequences(X_train)
                list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

                X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
                X_val   = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

                le = LabelBinarizer()
                le.fit(y_train)

                y_train = le.transform(y_train)[:,0:5]
                y_val = le.transform(y_val)[:,0:5]

                classifier = models.dl_model(model_type='CNN', max_features=WORD, dropout_rate=DROP, maxlen=LENGTH, classes=5, lr=RATE)
                if TRAIN:
                        classifier.fit(X_train, y_train, batch_size=128, epochs=4, shuffle=True, verbose=1)
                        classifier.save_weights(f'{TEST_PATH}/CNN_comments_v1.h5')
                else: 
                        classifier.load_weights(f'{TEST_PATH}/CNN_comments_v1.h5')
                probs = classifier.predict(X_val)

                assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                
                for i, cl in enumerate(le.classes_[le.classes_!='other']):
                    auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                    auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3) 

Epoch 1/4
9/9 [==============================] - 1s 36ms/step - loss: 1.2076 - acc: 0.3888
Epoch 2/4
9/9 [==============================] - 0s 28ms/step - loss: 0.9039 - acc: 0.6131
Epoch 3/4
9/9 [==============================] - 0s 27ms/step - loss: 0.8271 - acc: 0.6492
Epoch 4/4
9/9 [==============================] - 0s 27ms/step - loss: 0.6794 - acc: 0.6881


In [53]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.937,0.907,0.829,0.865,0.703,0.937,0.799,0.155,0.6,0.07
